In [1]:
import pandas as pd
import numpy as np
import scipy
from statsmodels.stats.weightstats import *

In [2]:
df = pd.read_csv('gene_high_throughput_sequencing.csv')
df.head(5)

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [3]:
df['Diagnosis'].value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

In [4]:
norm=df[df['Diagnosis']=='normal']
neoplasia=df[df['Diagnosis']=='early neoplasia']
cancer=df[df['Diagnosis']=='cancer']

In [5]:
p_values = pd.DataFrame(index = [0, 1], columns = df.columns[2:])

In [6]:
for column in p_values.columns: 
    p_values[column][0] = scipy.stats.ttest_ind(norm[column], neoplasia[column], equal_var = False).pvalue

In [7]:
for column in p_values.columns: 
    p_values[column][1] = scipy.stats.ttest_ind(neoplasia[column], cancer[column], equal_var = False).pvalue

In [8]:
n=(p_values.iloc[0] < 0.05).sum()

In [9]:
c=(p_values.iloc[1] < 0.05).sum()

In [10]:
gens=df.iloc[:,2:].columns

In [13]:
def write_answer(task,ans):
    with open("bio_" + str(task)+".txt", "w") as fout:
        fout.write(str(ans))

In [ ]:
write_answer(11, n)

In [ ]:
write_answer(12, c)

In [12]:
import statsmodels.stats.multitest as smm

In [159]:
def Fct(c, t) : # absolute value
    if t > c :
        r = t/c
    else :
        r =  c/t
    #print 'Ftc=%.2f' % r
    return r

In [207]:
def get_sugnificant(rejects, df1, df2):
    counter=0
    
    for i in range(len(rejects)):
        index=i+2
        c = df1.iloc[:, index].mean() 
        t = df2.iloc[:, index].mean()
        
        if rejects[i]==True:
            if pvals_corrected[i] < 0.025:
                if abs(Fct(c, t))>1.5:
                    counter+=1
    return counter

In [181]:
def foldchange_check (index, df1, df2):
    if df2.iloc[:, index].mean() > df1.iloc[:, index].mean():
        return df2.iloc[:, index].mean()/df1.iloc[:, index].mean()
    if df2.iloc[:, index].mean() <= df1.iloc[:, index].mean():
        return (df1.iloc[:, index].mean()/df2.iloc[:, index].mean())*-1

In [208]:
reject, pvals_corrected, alphacSidak, alphacBonf =  smm.multipletests(p_values.iloc[0], alpha=0.05, method='h')

In [209]:
n=get_sugnificant(reject, norm, neoplasia)

In [210]:
reject, pvals_corrected, alphacSidak, alphacBonf =  smm.multipletests(p_values.iloc[1], alpha=0.05, method='h')

In [211]:
c=get_sugnificant(reject, neoplasia, cancer)

In [117]:
norm.iloc[:, [5]].mean()

NOC2L    15.374351
dtype: float64

In [212]:
n, c

(2, 77)

In [213]:
write_answer(21, n)
write_answer(22, c)

In [214]:
reject, pvals_corrected, alphacSidak, alphacBonf =  smm.multipletests(p_values.iloc[0], alpha=0.05, method='fdr_tsbh')
n=get_sugnificant(reject, norm, neoplasia)

In [215]:
reject, pvals_corrected, alphacSidak, alphacBonf =  smm.multipletests(p_values.iloc[1], alpha=0.05, method='fdr_tsbh')
c=get_sugnificant(reject, neoplasia, cancer)

In [216]:
n, c

(4, 537)

In [219]:
c=538

In [220]:
write_answer(31, n)
write_answer(32, c)